In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, length, expr, when, current_timestamp, array
import psycopg2

# 1. Créer une session Spark
print("Création de la session Spark...")
spark = SparkSession.builder \
    .appName("PostgresToSpark") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.3.9.jar") \
    .getOrCreate()

# 2. Configurer les paramètres de connexion
print("Configuration des paramètres de connexion JDBC pour PostgreSQL...")
jdbc_url = "jdbc:postgresql://postgres:5432/DB_Mastodon"
properties = {
    "user": "fadi",
    "password": "fadi",
    "driver": "org.postgresql.Driver"
}

# 3. Lire les données depuis PostgreSQL
print("Lecture des données depuis la table 'Mostodon_SILVER' dans PostgreSQL...")
df = spark.read.jdbc(url=jdbc_url, table="Mostodon_SILVER", properties=properties)
print(f"Nombre de lignes lues : {df.count()}")

# 4. Nettoyer la colonne 'content' des balises HTML
print("Suppression des balises HTML dans la colonne 'content'...")
df_cleaned = df.withColumn("content", regexp_replace(col("content").cast("string"), "<[^>]+>", ""))

# 5. Calcul des nouvelles colonnes
print("Calcul des nouvelles colonnes...")
df_with_calculations = df_cleaned.withColumn("engagement_score", 
    (col("favourites_count") * 0.5) + 
    (col("reblogs_count") * 1.0) + 
    (col("replies_count") * 1.5)
).withColumn("content_length", length(col("content"))) \
 .withColumn("hashtags", expr("regexp_extract_all(content, '#(\\w+)')")) \
 .withColumn("contains_link", when(col("content").rlike("http[s]?://"), True).otherwise(False)) \
 .withColumn("normalized_engagement", 
    (col("favourites_count") + col("reblogs_count") + col("replies_count")) /
    ((current_timestamp().cast("long") - col("created_at").cast("long")) / 3600)
)

# 6. Convertir la colonne hashtags en tableau
df_with_calculations = df_with_calculations.withColumn("hashtags", 
    when(col("hashtags").isNotNull(), expr("array_distinct(hashtags)"))  # Crée un tableau distinct
)

# 7. Afficher les résultats calculés
df_with_calculations.show()

# 8. Connexion à PostgreSQL via psycopg2
print("Connexion à PostgreSQL avec psycopg2...")
conn = psycopg2.connect(
    host="postgres",
    database="DB_Mastodon",
    user="fadi",
    password="fadi"
)
cursor = conn.cursor()

# 9. Création de la table 'Mostodon_GOLD_bis' si elle n'existe pas déjà
print("Création de la table 'Mostodon_GOLD_bis' si elle n'existe pas déjà...")
create_table_query = '''
CREATE TABLE IF NOT EXISTS Mostodon_GOLD_bis (
    id BIGINT PRIMARY KEY,
    username TEXT NOT NULL,
    display_name TEXT NOT NULL,
    content TEXT NOT NULL,
    created_at TIMESTAMP NOT NULL,
    favourites_count INT NOT NULL,
    reblogs_count INT NOT NULL,
    replies_count INT NOT NULL,
    engagement_score FLOAT NOT NULL,
    content_length INT NOT NULL,
    hashtags TEXT[],
    contains_link BOOLEAN NOT NULL,
    normalized_engagement FLOAT NOT NULL
);
'''
cursor.execute(create_table_query)
conn.commit()

# 10. Lire les IDs existants dans la table Mostodon_GOLD_bis
existing_ids = spark.read.jdbc(url=jdbc_url, table="Mostodon_GOLD_bis", properties=properties) \
    .select("id") \
    .rdd.flatMap(lambda x: x).collect()

# 11. Filtrer le DataFrame pour exclure les enregistrements existants
df_filtered = df_with_calculations.filter(~col("id").isin(existing_ids))

# 12. Insertion des données filtrées dans la table Mostodon_GOLD_bis
print("Insertion des données calculées dans la table 'Mostodon_GOLD_bis' via Spark JDBC...")
df_filtered.write.jdbc(url=jdbc_url, table="Mostodon_GOLD_bis", mode="append", properties=properties)
print("Insertion terminée.")

# 13. Compter le nombre total d'entrées dans la table 'Mostodon_GOLD_bis'
print("Calcul du nombre total d'entrées dans la table 'Mostodon_GOLD_bis'...")
cursor.execute("SELECT COUNT(*) FROM Mostodon_GOLD_bis;")
count = cursor.fetchone()[0]
print(f"Nombre total d'entrées dans la table 'Mostodon_GOLD_bis' : {count}")

# 14. Fermer le curseur et la connexion PostgreSQL
cursor.close()
conn.close()
print("Connexion à PostgreSQL fermée.")

# 15. Fermer la session Spark
print("Fermeture de la session Spark...")
spark.stop()
print("Session Spark fermée.")


Création de la session Spark...
Configuration des paramètres de connexion JDBC pour PostgreSQL...
Lecture des données depuis la table 'Mostodon_SILVER' dans PostgreSQL...
Nombre de lignes lues : 747
Suppression des balises HTML dans la colonne 'content'...
Calcul des nouvelles colonnes...
+------------------+--------------------+------------------------------+-------------------------------------+--------------------+----------------+-------------+-------------+----------------+--------------+--------+-------------+---------------------+
|                id|            username|                  display_name|                              content|          created_at|favourites_count|reblogs_count|replies_count|engagement_score|content_length|hashtags|contains_link|normalized_engagement|
+------------------+--------------------+------------------------------+-------------------------------------+--------------------+----------------+-------------+-------------+----------------+---------